In [17]:
import torch
import torchvision
from torchvision import datasets,models,transforms
import numpy as np
from matplotlib import pyplot as plt
import os

import argparse
parser = argparse.ArgumentParser()

parser.add_argument('-l','--filelist', nargs='+', help='<Required> Set flag', required=True)

args = parser.parse_args()



T = {'train': transforms.Compose([ transforms.RandomHorizontalFlip(),
                                   transforms.ToTensor(),
                                   transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))       
                                 ]) ,
      
    'test': transforms.Compose([ transforms.RandomHorizontalFlip(),
                                   transforms.ToTensor(),
                                   transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))       
                                 ]) ,
      
    'valid': transforms.Compose([ transforms.RandomHorizontalFlip(),
                                   transforms.ToTensor(),
                                   transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))       
                                 ]) 
    }

datasets={x: datasets.ImageFolder( os.path.join(root,x), T[x] ) for x in args.filelist }
dataloaders={x : torch.utils.data.DataLoader(datasets[x] , shuffle=True, batch_size=4 , num_workers=2) for x in args.filelist}
datalength= { x : len(datasets_reqd[x]) for x in args.filelist}
device=torch.device('cuda:0' if torch.cuda.is_available() else "cpu")
class_names=datasets_reqd['train'].classes
print(classes_names)

usage: ipykernel_launcher.py [-h] -l FILELIST [FILELIST ...]
ipykernel_launcher.py: error: the following arguments are required: -l/--filelist


SystemExit: 2

/home/ashish95/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3275: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [14]:
import torch
import torchvision
from torchvision import datasets,models,transforms
import numpy as np
from matplotlib import pyplot as plt
import os

import argparse


In [16]:
def training( model , scheduler , optimizer, criterion):
    best_acc=0
    best_model_wts= copy.deepcopy(model.state_dict())
    for epoch in range(num_epochs):
        
        for phase in args.filelist:
            if phase == 'train':
                scheduler.step()
                model.train()
            else:
                model.eval()
            running_loss=0
            running_corrects=0
            for images,labels in dataloaders[phase]:
            
                images=images.to(device)
                labels=labels.to(device)
                optimizer.zero_grad()
                with torch.set_grad_enabled(phase=='Train'):
                    output = model(images)
                    _,predictions=torch.max(outputs,1)
                    loss = criterion(outputs,labels)
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                running_loss = loss.item()*images.size(0)
                running_corrects= torch.sum(predictions==labels.data)
            epoch_loss=running_loss/datalength[phase]
            epoch_acc=running_corrects.double()/datalength[phase]
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            
            if phase=='val' and epoch_acc>best_acc:
                best_acc=epoch_acc
                best_model_wts=copy.deepcopy(model.state_dict())
    model.load_state_dict(best_model_wts)    
    return model

                

In [ ]:
import torch.nn as nn
model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 2)

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)



model_conv = train_model(model_conv, criterion, optimizer_ft,exp_lr_scheduler, num_epochs=25)

In [18]:
#!/usr/bin/env python
# coding: utf-8

# In[ ]:


import torch
import torchvision
from torchvision import datasets,models,transforms
import numpy as np
from matplotlib import pyplot as plt
import os
from skimage import io
import pathlib


from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

"""
import argparse
parser = argparse.ArgumentParser()

parser.add_argument('-l','--filelist', nargs='+', help='<Required> Set flag', required=True)

args = parser.parse_args()
"""

import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import time
import copy


ngpu=2

root='/ssd_scratch/cvit/ashish/kidney/'

T = {'train': transforms.Compose([ transforms.RandomResizedCrop(224),
                                   transforms.RandomHorizontalFlip(),
                                   transforms.ToTensor(),
                                   transforms.Normalize([0.596, 0.436, 0.586], [0.2066, 0.240, 0.186])       
                                 ]) ,
      
    'val': transforms.Compose([   transforms.Resize(224),
                                   transforms.RandomHorizontalFlip(),
                                   transforms.ToTensor(),
                                   transforms.Normalize([0.596, 0.436, 0.586], [0.2066, 0.240, 0.186])       
                                 ]) ,
      
    'test': transforms.Compose([ transforms.Resize(224),
                                   transforms.ToTensor(),
                                   transforms.Normalize([0.596, 0.436, 0.586], [0.2066, 0.240, 0.186])       
                                 ]) 
    }

datasets={x: datasets.ImageFolder( os.path.join(root,x), T[x] ) for x in ['test'] }
dataloaders={x : torch.utils.data.DataLoader(datasets[x] , shuffle=True, batch_size=128 , num_workers=4) for x in ['test']}
datalength= { x : len(datasets[x]) for x in ['test']}
print(datalength['test'])
device=torch.device('cuda:0' if torch.cuda.is_available() else "cpu")
class_names=datasets['test'].classes
print(class_names)


"""
def imgshow(img):
    im=img.numpy().transpose((1,2,0))
    mean = np.array([0.596, 0.436, 0.586])
    std = np.array([0.2066, 0.240, 0.186])
    im=im*std + mean
    plt.imsave('images.jpg',im)




images,labels= iter(dataloaders['train']).next()
images=torchvision.utils.make_grid(images)
imgshow(images)
"""
lst=['test']
acc_list=[]
loss_list=[]
epoch_list=[]
#lst=lst.sort()
print(lst)
def train_model(model):
    since=time.time()
    best_model_wts=copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    for epoch in range(num_epochs):
        print('-'*10 + ' Epoch {}'.format(epoch) + '-'*10, flush=True)
            
        for phase in lst:
            print(phase)
            if phase=='train':
                scheduler.step()
                model.train()
            else:
                model.eval()
            running_loss=0
            running_corrects=0
            for data in dataloaders[phase]:
                k=k+1;
                if(k%100 == 0):
                    print(k/100)

                # get the inputs
                inputs, labels, idx = data

                # wrap them in Variable
                if use_gpu:
                    inputs = Variable(inputs.cuda())
                    labels = Variable(labels.cuda())
                else:
                    inputs = Variable(inputs)
                    labels = Variable(labels)

                # zero the parameter gradients

                #bs, ncrops, c, h, w = inputs.size()
                #inp_chg = inputs.view(-1, c, h, w)
                outputs = model(inputs)	
                #outputs = F.tanh(outputs)
                #outputs = (outputs+1)/2

                #outputs_avg = outputs.view(bs, ncrops, -1).mean(1)
                #print(outputs)
                _, preds = torch.max(outputs.data, 1)
                f = open("/home/ashishmenon/openslide_WSI/synth_pretraining/test_doc.txt","a+")
                f.write(str(torch.max(outputs).item()) + "\n" )
                f.close()

                f = open("/home/ashishmenon/openslide_WSI/synth_pretraining/test_classes.txt","a+")
                f.write(str(preds.item()) + "\n" )
                f.close()

                running_corrects += torch.sum(preds == labels.data)

                #print('running loss',running_loss)
            epoch_acc = (running_corrects.double()) / dataset_sizes[phase]

            print('{} resnet-18-Acc: {:.4f}'.format(
                    phase, epoch_acc))
            print(running_corrects)
            print(dataset_sizes[phase])


        print()

    time_elapsed = time.time() - since
    print('testing complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))


    return model
   


d='/home/ashishmenon/openslide_WSI/synth_pretraining/trained.pth'

fd=pathlib.Path(d)


model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 2)


if (fd.exists ()):
    print('Model loading')
    model_ft = model_ft.to(device)
    if (device.type == 'cuda') and (ngpu > 1):
        model_ft = nn.DataParallel(model_ft, list(range(ngpu)))
    model_ft.load_state_dict(torch.load(d))
    print('Model loaded')


model_conv = train_model(model_ft)

['train', 'val']

In [19]:
import torch

In [36]:
x=torch.rand(4,2,3)
print(x)
torch.max(x.data,1)


tensor([[[0.6114, 0.8601, 0.7593],
         [0.4912, 0.6661, 0.0395]],

        [[0.0482, 0.8588, 0.5801],
         [0.8671, 0.2377, 0.8419]],

        [[0.5581, 0.2066, 0.7577],
         [0.8969, 0.0351, 0.0335]],

        [[0.2962, 0.3896, 0.1435],
         [0.4134, 0.8845, 0.0913]]])


torch.return_types.max(
values=tensor([[0.6114, 0.8601, 0.7593],
        [0.8671, 0.8588, 0.8419],
        [0.8969, 0.2066, 0.7577],
        [0.4134, 0.8845, 0.1435]]),
indices=tensor([[0, 0, 0],
        [1, 0, 1],
        [1, 0, 0],
        [1, 1, 0]]))

In [81]:
classes={ 0:'cancer' , 1:'normal'}

_, preds = torch.max(x.data, 1)
preds.view(-1).numpy()
preds
xy=list((preds.view(-1).numpy()))
for i in range(len(xy)):
    print(xy[i])
    
for i in range(len(xy)):
    f = open("./test.txt","a+")
    f.write(str(xy[i])+ "\n" )
    f.close()

0
0
0
1
0
1
1
0
0
1
1
0
